In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
qsmax = 0.50976  # max substrate uptake rate [gS/gX/h]
K_s = 0.079651  # Ks value for glucose [g/L]
# Define model constant
c_glu = 350  # conc. of glucose in Feed [g/L]

Yxs = 0.37405  # Yield coefficient for Glucose [gX/gS]
Ypx = 1369.8477  # Yield coefficient for Product [gP/gX]
Yo2x = 0.038  # mol/g
Yco2x = 0.037  # mol/g

# p.kd = 0.001  # constant - cell death[1/h]?
lag = 0.0040574

maxpO2 = 0.2095  # for %dO2 calculation 0.2095   0.605
# p.cO2max = 0.000239  # mol/L
# p.cCO2max = 0.00116  # mol/L

# p.kLaO2 = 300  # h-1
alpha1 = 0.8303  # 0.7117
alpha2 = 0.2956  # 0.0657
beta = 0.4575  # 0.4298
K_value = 0.0123  # 0.0215
stirrerdiameter = 0.046  # m
crossarea = 0.01  # m2

y_CO2_in = 0.032  # %0.032        0.020   0.016
y_O2_in = 20.95  # %20.95        50.03   60.05
y_O2_O2in = 99.9

epsilon = 0.2
T_zero = 298.15  # K
H_CO2_standard = 3.4e-2  # mol/(L atm) - später dann 1/x um auf L atm/ mol zu kommen
deltaH_R_CO2 = 2400  # von Sven übernommen (calc_offgas_corrSDA)

H_O2_standard = 1.3e-3
deltaH_R_O2 = 1700  # von Sven übernommen (calc_offgas_corrSDA)

gas_constant = 8.314462  # kg*m2/s2/mol/K

kLa_factor = 0.80

# p.kplus = 211.32  # [1/h];Kappelmann
# p.kminus = 19.152  # [1/h];Kappelmann
k1 = 129.6
k2 = 41760000
Kw = 10**(-13.8)

# p.a = 0.0123  # exponential correlation exponent for viscosity (data-driven!)
#Note: The commented lines involving p.kplus, p.kminus, and p.a have been excluded as there is no value assigned to them in the given MATLAB code. You can uncomment and assign appropriate values if needed.

In [ ]:
# Define model equations

# Substrate uptake rate qs
def qs(x, IPTG):
    return qsmax * (x[2] / (x[2] + K_s)) * (1 / (1 + IPTG * (np.exp(x[10] * lag))))
# Biomass growth rate
def qx(x, IPTG):
    return qs(x, IPTG) * Yxs
# convert T in Celsius to Kelvin
def cel_to_kel(T):
    return (T+273.15)
# Temperature dependence of Henry constant - O2
def H_O2(T):
    return 1.0/(H_O2_standard*np.exp(deltaH_R_O2*(1/(cel_to_kel(T) - (1/T_zero)))))
# Temperature dependence of Henry constant - CO2
def H_CO2(T):
    return 1/(H_CO2_standard*np.exp(deltaH_R_CO2*(1/(cel_to_kel(T) - (1/T_zero)))))
# Airflow import + convert from NL/min --> m3/h
def F_air(f_air):
    return f_air * 60/1000
# Oxygenflow import + convert from NL/min --> m3/h
def F_o2(f_o2):
    return f_o2 * 60/1000
# sum of gas flow
def F_gas(f_o2, f_air):
    return F_o2(f_o2) + F_air(f_air)
# pressure --> umrechnen in pascal
def pressure_pascal(pressure):
    return pressure * 100000
# gas hold up - V_air
def V_air(epsilon, x):
    return epsilon * x[0]

def yO2_out(x, pressure):
    return ((x[5]*101325 / pressure_pascal(pressure))*100)
def yCO2_out(x, pressure):
    return ((x[7]*101325 / pressure_pascal(pressure))*100)

def K1(T):
    return (np.exp(-11.582 -(918.9/cel_to_kel(T))))
def K2(T, Kw):
    return (K1(T) / Kw)

def kLaO2(K_value, stirrerspeed, alpha1, stirrerdiameter, alpha2, f_air, f_o2, crossarea, beta):
    return (K_value*((stirrerspeed*60)**alpha1)*(stirrerdiameter**alpha2)*((F_gas(f_air,f_o2)/crossarea)**beta))
# in mol/(Lh)
def OTR(K_value, stirrerspeed, alpha1, stirrerdiameter, alpha2, f_air, f_o2, crossarea, beta, x, T):
    return (kLaO2(K_value, stirrerspeed, alpha1, stirrerdiameter, alpha2, f_air, f_o2, crossarea, beta)*(x[5]/(H_O2(T)-x[4])))
# in mol/(Lh)
def OUR(x, IPTG):
    return (qx(x, IPTG) * Yo2x *x[1])
# OUR mol/h
def OUR2(x, IPTG, volume):
    return (OUR(x, IPTG) * volume * (-1))
def dO2(x, T):
    return (x[4] * 100 /(maxpO2/H_O2(T)))

def kLaCO2(kLa_factor, K_value, stirrerspeed, alpha1, stirrerdiameter, alpha2, f_air, f_o2, crossarea, beta):
    return (kLaO2(K_value, stirrerspeed, alpha1, stirrerdiameter, alpha2, f_air, f_o2, crossarea, beta)*kLa_factor)
# CTR mol/(Lh)
def CTR(kLa_factor, K_value, stirrerspeed, alpha1, stirrerdiameter, alpha2, f_air, f_o2, crossarea, beta, x, T):
    return ((-kLaCO2(kLa_factor, K_value, stirrerspeed, alpha1, stirrerdiameter, alpha2, f_air, f_o2, crossarea, beta) * x[7]/(H_CO2(T)-x[6])))


def dO2_molperL(x):
    OTR = kLaO2 * (x[6] / H_O2 - x[5])
    OUR = qx(x, IPTG) * YO2X * x[2]
    return OTR - OUR

def pO2(x):
    pO2_in = p.y_O2_in.Value / 100 * p.Fair.Value / p.R.Value / EQ.T
    pO2_O2in = p.y_O2_O2in.Value / 100 * p.FO2.Value / p.R.Value / EQ.T
    return (((-OTR * x[1]) - (p.Fgas.Value * x[6] * 101325 / p.R.Value / EQ.T) + (p.p.Value * pO2_in) + (p.p.Value * pO2_O2in)) * p.R.Value * EQ.T / (Vair / 1000)) / 101325
